# 🛍️ Gender-Based Sales Trends in Supermarket Transactions  
### Exploring Consumer Behavior and Ethical Considerations in Business Intelligence

> _How do product preferences and average purchase values differ by gender — and what are the ethical implications of using this data in decision-making?_


### Set up the Notebook

In [9]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import pandas as pd



### Connect to SQLite Database

In [10]:
# Connect to your SQLite DW
conn = sqlite3.connect(r"C:\Users\beths\Documents\Northwest_Missouri_Analytics\BI_Analytics\supermarket_sales\data\dw\supermarket_dw.db")


### Load and Explore Data

In [11]:
# Load tables
df_customers = pd.read_sql("SELECT * FROM customer", conn)
df_products = pd.read_sql("SELECT * FROM product", conn)
df_sales = pd.read_sql("SELECT * FROM sale", conn)

# Merge tables into one analysis DataFrame
df = df_sales.merge(df_customers, on="customer_id").merge(df_products, on="product_id")
df["datetime"] = pd.to_datetime(df["datetime"])
df["month"] = df["datetime"].dt.to_period("M").astype(str)

df.head()


,sale_id,customer_id,product_id,quantity_sold,total,branch,city,datetime,rating,gender,payment_method,product_line,unit_price,tax_pct,month
0,750-67-8428,750-67-8428,750-67-8428,7,548.9715,A,Yangon,2019-01-05 13:08:00,9.1,Female,Ewallet,Health and beauty,74.69,26.1415,2019-01
1,226-31-3081,226-31-3081,226-31-3081,5,80.2200,C,Naypyitaw,2019-03-08 10:29:00,9.6,Female,Cash,Electronic accessories,15.28,3.8200,2019-03
2,631-41-3108,631-41-3108,631-41-3108,7,340.5255,A,Yangon,2019-03-03 13:23:00,7.4,Male,Credit card,Home and lifestyle,46.33,16.2155,2019-03
3,123-19-1176,123-19-1176,123-19-1176,8,489.0480,A,Yangon,2019-01-27 20:33:00,8.4,Male,Ewallet,Health and beauty,58.22,23.2880,2019-01
4,373-73-7910,373-73-7910,373-73-7910,7,634.3785,A,Yangon,2019-02-08 10:37:00,5.3,Male,Ewallet,Sports and travel,86.31,30.2085,2019-02
